In [6]:
import cv2
import dlib
import numpy as np
import os
from eulerian_magnification import eulerian_magnification as em

# Initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to get facial landmarks
def get_landmarks(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    if len(rects) > 0:
        return predictor(gray, rects[0])
    return None

# Function to remove eyes and background
def remove_eyes_background(frame, landmarks):
    mask = np.zeros_like(frame)
    points = [landmarks.part(i) for i in range(68) if i < 36 or (i >= 42 and i < 48)]
    points = np.array([(point.x, point.y) for point in points])
    cv2.fillConvexPoly(mask, points, (1, 1, 1))
    return frame * mask

# Function to apply motion magnification using Eulerian Video Magnification
def motion_magnify(input_path, output_path, alpha=20, lambda_c=16, fl=0.4, fh=3.0, sampling_rate=30):
    em.magnify_video(input_path, output_path, alpha=alpha, lambda_c=lambda_c, fl=fl, fh=fh, sampling_rate=sampling_rate)

# Function to generate MMSTR maps
def generate_mmstr(video_path, n_blocks):
    cap = cv2.VideoCapture(video_path)
    frames = []
    landmarks_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        landmarks = get_landmarks(frame)
        if landmarks:
            frame = remove_eyes_background(frame, landmarks)
            landmarks_list.append(landmarks)
        frames.append(frame)
    cap.release()

    # Apply motion magnification
    magnified_video_path = "magnified_" + os.path.basename(video_path)
    motion_magnify(video_path, magnified_video_path)

    cap = cv2.VideoCapture(magnified_video_path)
    mmstr_maps = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        h, w, _ = frame.shape
        block_h = h // n_blocks
        block_w = w // n_blocks
        for i in range(n_blocks):
            for j in range(n_blocks):
                block = frame[i * block_h:(i + 1) * block_h, j * block_w:(j + 1) * block_w]
                r_mean = np.mean(block[:, :, 0])
                g_mean = np.mean(block[:, :, 1])
                b_mean = np.mean(block[:, :, 2])
                if len(mmstr_maps) <= i * n_blocks + j:
                    mmstr_maps.append([[], [], []])
                mmstr_maps[i * n_blocks + j][0].append(r_mean)
                mmstr_maps[i * n_blocks + j][1].append(g_mean)
                mmstr_maps[i * n_blocks + j][2].append(b_mean)
    cap.release()

    return mmstr_maps

# Example usage
video_path = "path_to_video.mp4"
n_blocks = 4  # Number of non-overlapping ROI blocks per dimension (e.g., 4x4)
mmstr_maps = generate_mmstr(video_path, n_blocks)

# Visualize MMSTR maps
import matplotlib.pyplot as plt

for idx, mmstr in enumerate(mmstr_maps):
    plt.figure(figsize=(10, 2))
    plt.title(f"Block {idx + 1}")
    plt.plot(mmstr[0], color='r', label='Red')
    plt.plot(mmstr[1], color='g', label='Green')
    plt.plot(mmstr[2], color='b', label='Blue')
    plt.legend()
    plt.show()


AttributeError: 'function' object has no attribute 'magnify_video'

In [ ]:
import cv2
import dlib
import numpy as np
import os
from eulerian_magnification import magnify

# Initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Function to get facial landmarks
def get_landmarks(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    if len(rects) > 0:
        return predictor(gray, rects[0])
    return None

# Function to remove eyes and background
def remove_eyes_background(frame, landmarks):
    mask = np.zeros_like(frame)
    points = [landmarks.part(i) for i in range(68) if i < 36 or (i >= 42 and i < 48)]
    points = np.array([(point.x, point.y) for point in points])
    cv2.fillConvexPoly(mask, points, (1, 1, 1))
    return frame * mask

# Function to apply motion magnification using Eulerian Video Magnification
def motion_magnify(input_path, output_path, alpha=20, lambda_c=16, fl=0.4, fh=3.0, sampling_rate=30):
    magnify(input_path, output_path, alpha=alpha, lambda_c=lambda_c, fl=fl, fh=fh, samplingRate=sampling_rate)

# Function to generate MMSTR maps
def generate_mmstr(video_path, n_blocks):
    cap = cv2.VideoCapture(video_path)
    frames = []
    landmarks_list = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        landmarks = get_landmarks(frame)
        if landmarks:
            frame = remove_eyes_background(frame, landmarks)
            landmarks_list.append(landmarks)
        frames.append(frame)
    cap.release()

    # Apply motion magnification
    magnified_video_path = "magnified_" + os.path.basename(video_path)
    motion_magnify(video_path, magnified_video_path)

    cap = cv2.VideoCapture(magnified_video_path)
    mmstr_maps = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        h, w, _ = frame.shape
        block_h = h // n_blocks
        block_w = w // n_blocks
        for i in range(n_blocks):
            for j in range(n_blocks):
                block = frame[i * block_h:(i + 1) * block_h, j * block_w:(j + 1) * block_w]
                r_mean = np.mean(block[:, :, 0])
                g_mean = np.mean(block[:, :, 1])
                b_mean = np.mean(block[:, :, 2])
                if len(mmstr_maps) <= i * n_blocks + j:
                    mmstr_maps.append([[], [], []])
                mmstr_maps[i * n_blocks + j][0].append(r_mean)
                mmstr_maps[i * n_blocks + j][1].append(g_mean)
                mmstr_maps[i * n_blocks + j][2].append(b_mean)
    cap.release()

    return mmstr_maps

# Example usage
video_path = r"CELEB\REAL\id59_0006.mp4"
n_blocks = 4  # Number of non-overlapping ROI blocks per dimension (e.g., 4x4)
mmstr_maps = generate_mmstr(video_path, n_blocks)

# Visualize MMSTR maps
import matplotlib.pyplot as plt

for idx, mmstr in enumerate(mmstr_maps):
    plt.figure(figsize=(10, 2))
    plt.title(f"Block {idx + 1}")
    plt.plot(mmstr[0], color='r', label='Red')
    plt.plot(mmstr[1], color='g', label='Green')
    plt.plot(mmstr[2], color='b', label='Blue')
    plt.legend()
    plt.show()
